In [32]:
import re
import string
import nltk
import collections
import numpy as np
import pandas as pd
from nltk import word_tokenize
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score, matthews_corrcoef
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [33]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.4.0
Running on TPU  ['10.5.33.106:8470']


INFO:tensorflow:Initializing the TPU system: grpc://10.5.33.106:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.5.33.106:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [34]:
vocab_size=10000
maxlen= 50
output_dim = 32

def read_file(name):
  """
  Reads csv file and keeps row with acceptables class values
  """
  df = pd.read_csv(name)
  df = df[df['Class'].isin(['0','1'])]
  return df.dropna()

def split_data(x, y):
  """
  Split into train test
  """
  y_data = np.array([int(i) for i in y])
  return train_test_split(x_data, y_data, test_size = 0.3)

stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

all_words = []
def sanitize_text(text):
  """
  Cleans tweet data by removing mentions, http links and retweet tags
  """
  try:
      # replace retweet tags
      text = re.sub('^(RT @\w+)', '', text)
      # replace website URLs
      text = re.sub('(https://)([A-Za-z0-9./]+)', '', text)
      # replace mentions
      text = re.sub('(@\w+)', '', text)
      text_lc = "".join([word.lower() for word in text if word not in string.punctuation])
      text_rc = re.sub('[0-9]+', '', text_lc)
      tokens = word_tokenize(text_rc)
      text = [ps.stem(word) for word in tokens if word not in stopword]
      all_words.extend(text)
  except Exception as e:
      print(e)
  return text

def clean_data(df):
  """
  Creates new column with cleaned text and keeps only top 10000 most frequent words
  """
  df['cleaned_text'] = df['Text'].apply(sanitize_text)
  keys = dict(collections.Counter(all_words).most_common(vocab_size)).keys()

  def keep_top(text):
    text = [i for i in text if i in keys]
    return " ".join(text)


  df['cleaned_text'] = df['cleaned_text'].apply(keep_top)
  return df

def encode_data(df, x):
  """
  Integer encodes words in text to prepare data for embeddings
  """
  encod_corp=[]
  for i,doc in enumerate(df[x].tolist()):
      encod_corp.append(one_hot(doc,vocab_size))
  pad_corp=pad_sequences(encod_corp,maxlen=maxlen,padding='pre',value=0.0)
  return pad_corp

def create_model():
  """
  Create model as mentioned in image
  """
  model = Sequential()
  model.add(layers.Input(shape=(maxlen,), dtype='float64'))
  model.add(layers.Embedding(input_dim=vocab_size,output_dim=output_dim,input_length=maxlen))

  model.add(layers.Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(50,32)))
  model.add(layers.MaxPooling1D(pool_size=2))

  model.add(layers.Bidirectional(layers.LSTM(256, return_sequences=True)))
  model.add(layers.Dropout(0.5))

  model.add(layers.Bidirectional(layers.LSTM(256)))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(1024, activation='relu'))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(1, activation='sigmoid'))

  adam = Adam(learning_rate=0.0005)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

def run_and_evaluate(epochs=1, batchsize=512):
  """
  trains model and evaluates on given metrics
  """
  model.fit(x_train, y_train, epochs=epochs, batch_size=batchsize, verbose=1)
  y_pred = model.predict(x_test)
  y_pred = [round(i[0]) for i in y_pred]
  acc = accuracy_score(y_test, y_pred)
  print(f"Accuracy score: {acc}")
  mcc = matthews_corrcoef(y_test, y_pred)
  print(f"Maththews correlation coefficient: {mcc}")
  cm = confusion_matrix(y_test, y_pred)
  print(f"Confusion Matrix:\n{cm}")

In [35]:
df = read_file("Colab Notebooks/data/CVAssignmentDataset.csv")
df_cleaned = clean_data(df)
x_data = encode_data(df_cleaned, 'cleaned_text')
x_train, x_test, y_train, y_test = split_data(x_data, df_cleaned['Class'].tolist())
model = create_model()

In [36]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 32)            320000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 32)            1056      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 25, 32)            0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 25, 512)           591872    
_________________________________________________________________
dropout_8 (Dropout)          (None, 25, 512)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 512)               1574912   
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)              

In [37]:
run_and_evaluate(12, 512)

Epoch 1/10
119/119 [==============================] - 43s 309ms/step - loss: 0.6697 - accuracy: 0.5543
Epoch 2/10
119/119 [==============================] - 37s 311ms/step - loss: 0.5062 - accuracy: 0.7506
Epoch 3/10
119/119 [==============================] - 37s 310ms/step - loss: 0.4640 - accuracy: 0.7753
Epoch 4/10
119/119 [==============================] - 37s 310ms/step - loss: 0.4458 - accuracy: 0.7853
Epoch 5/10
119/119 [==============================] - 37s 310ms/step - loss: 0.4333 - accuracy: 0.7924
Epoch 6/10
119/119 [==============================] - 37s 310ms/step - loss: 0.4186 - accuracy: 0.8015
Epoch 7/10
119/119 [==============================] - 37s 310ms/step - loss: 0.3948 - accuracy: 0.8139
Epoch 8/10
119/119 [==============================] - 37s 314ms/step - loss: 0.3743 - accuracy: 0.8230
Epoch 9/10
119/119 [==============================] - 37s 310ms/step - loss: 0.3852 - accuracy: 0.8204
Epoch 10/10
119/119 [==============================] - 37s 309ms/step - l